In [1]:
%pip install azure-keyvault-secrets
%pip install azure-identity
%pip install azure-keyvault-secrets
%pip install torch
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.models import densenet201
from azureml.core import Workspace, Dataset, Experiment, Run

from __future__ import print_function, division
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import numpy as np
import time
import copy
import argparse
import mlflow
from azure.identity import AzureCliCredential
from azure.keyvault.secrets import SecretClient
from azure.storage.filedatalake import DataLakeServiceClient
from azure.identity import AzureCliCredential
from azure.keyvault.secrets import SecretClient

from azure.ai.ml import command
from azure.ai.ml import Input

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
key_vault_name = 'serengetikeyvaultqezfaqk'
key_vault_url = f'https://{key_vault_name}.vault.azure.net/'
pool_conn_secret_name = 'DedicatedPool-odbc-ConnectionString'
adls_conn_secret_name = 'ADLS-ConnectionString'

credential = AzureCliCredential()
secret_client = SecretClient(vault_url=key_vault_url, credential = credential)

adls_connection_string = secret_client.get_secret(adls_conn_secret_name).value

In [4]:
service_client = DataLakeServiceClient.from_connection_string(adls_connection_string)

file_system_name = "snapshot-serengeti"
file_path = "images/train"
file_system_client = service_client.get_file_system_client(file_system=file_system_name)
paths = file_system_client.get_paths(path=file_path)

for path in paths:
    print(path.name + "\n")

images/train/baboon

images/train/baboon/S3_P04_R11_IMAG0017.JPG

images/train/baboon/S3_P04_R11_IMAG0018.JPG

images/train/baboon/S3_P04_R11_IMAG0019.JPG

images/train/baboon/S7_I01_R1_IMAG0011.JPG

images/train/baboon/S7_I01_R1_IMAG0012.JPG

images/train/baboon/S7_I01_R1_IMAG0013.JPG

images/train/baboon/S7_I01_R1_IMAG0014.JPG

images/train/baboon/S7_I01_R1_IMAG0015.JPG

images/train/baboon/S7_I01_R1_IMAG0016.JPG

images/train/buffalo

images/train/buffalo/S10_I11_R2_IMAG0283.JPG

images/train/buffalo/S2_D05_R1_PICT2690.JPG

images/train/buffalo/S2_D05_R1_PICT2691.JPG

images/train/buffalo/S2_D05_R1_PICT2692.JPG

images/train/buffalo/S2_D05_R1_PICT2693.JPG

images/train/buffalo/S2_D05_R1_PICT2694.JPG

images/train/buffalo/S2_D05_R1_PICT2695.JPG

images/train/buffalo/S2_D05_R1_PICT2696.JPG

images/train/buffalo/S2_D05_R1_PICT2697.JPG

images/train/buffalo/S2_D05_R1_PICT2698.JPG

images/train/buffalo/S2_D05_R1_PICT2699.JPG

images/train/buffalo/S2_D05_R1_PICT2700.JPG

images/train/buff

In [5]:
import azureml.core

print("Azure ML SDK Version: ", azureml.core.VERSION)


Azure ML SDK Version:  1.48.0


In [6]:
# Get Azure Machine Learning workspace
ws = Workspace.from_config()

# Get Azure Machine Learning run context
run = Run.get_context()

In [7]:
# Get the data folder location
data_folder =  service_client.get_file_system_client(file_system=file_system_name)

# Define transforms for the data
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [21]:
# Create a job environment
curated_env_name = "azureml://registries/azureml/environments/AzureML-ACPT-pytorch-1.11-py38-cuda11.3-gpu/versions/11"
# curated_env_name = "azureml://locations/westeurope/workspaces/37a5255b-7545-48d3-938e-76434a1ab7be/environments/serengeti/versions/1"

In [22]:
job = command(
    inputs=dict(
        num_epochs=30, learning_rate=0.001, momentum=0.9, output_dir="./outputs"
    ),
    compute="bethanyjep1",
    environment=curated_env_name,
    code="./src",  # location of source code
    command="python pytorch_train.py --num_epochs ${{inputs.num_epochs}} --output_dir ${{inputs.output_dir}}",
    experiment_name="pytorch-serengeti",
    display_name="pytorch-serengeti-image",
)

In [23]:
# Get a handle to the workspace
# Handle to the workspace
from azure.ai.ml import MLClient
ml_client = MLClient(
    credential=credential,
    subscription_id="a1a27566-3e3c-42d7-a372-692095cd8521",
    resource_group_name="SerengetiDataLab",
    workspace_name="SerengetiAMLqezfaqkji2jnm",
)

In [24]:
ml_client.jobs.create_or_update(job)

Experiment,Name,Type,Status,Details Page
pytorch-serengeti,jolly_leg_sybbh7g37y,command,Starting,Link to Azure Machine Learning studio


In [ ]:
# tuning hyperparameters
from azure.ai.ml.sweep import Uniform

# we will reuse the command_job created before. we call it as a function so that we can apply inputs
job_for_sweep = job(
    learning_rate=Uniform(min_value=0.0005, max_value=0.005),
    momentum=Uniform(min_value=0.9, max_value=0.99),
)

In [ ]:
from azure.ai.ml.entities import AmlCompute

gpu_compute_taget = "bethanyjep1"


In [ ]:
from azure.ai.ml.sweep import BanditPolicy

sweep_job = job_for_sweep.sweep(
    compute="bethanyjep1",
    sampling_algorithm="random",
    primary_metric="best_val_acc",
    goal="Maximize",
    max_total_trials=8,
    max_concurrent_trials=4,
    early_termination_policy=BanditPolicy(
        slack_factor=0.15, evaluation_interval=1, delay_evaluation=10
    ),
)

In [ ]:
returned_sweep_job = ml_client.create_or_update(sweep_job)

# stream the output and wait until the job is finished
ml_client.jobs.stream(returned_sweep_job.name)

# refresh the latest status of the job after streaming
returned_sweep_job = ml_client.jobs.get(name=returned_sweep_job.name)

In [ ]:
from azure.ai.ml.entities import Model

if returned_sweep_job.status == "Completed":

    # First let us get the run which gave us the best result
    best_run = returned_sweep_job.properties["best_child_run_id"]

    # lets get the model from this run
    model = Model(
        # the script stores the model as "outputs"
        path="azureml://jobs/{}/outputs/artifacts/paths/outputs/".format(best_run),
        name="run-model-example",
        description="Model created from run.",
        type="custom_model",
    )

else:
    print(
        "Sweep job status: {}. Please wait until it completes".format(
            returned_sweep_job.status
        )
    )